<a href="https://colab.research.google.com/github/sarfrazahmd982/AI-Generated-text-detection-in-articles-English-/blob/main/BERT_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Imports and PIPs***

In [ ]:
!pip install transformers torch pandas scikit-learn

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import Dataset

***PRE-PROCESSING***

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

***BERT - MODEL ***

In [ ]:
train_df = pd.read_csv('labelled_train_set (1).csv')
test_df = pd.read_csv('unlabelled_test2 (1).csv')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
label_mapping = {'Human-written': 0, 'AI-generated': 1}
train_df['Type'] = train_df['Type'].map(label_mapping)

In [ ]:
X = train_df['Article'].tolist()
y = train_df['Type'].tolist()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
MAX_LEN = 256

In [ ]:
train_dataset = TextDataset(X_train, y_train, tokenizer, MAX_LEN)
val_dataset = TextDataset(X_val, y_val, tokenizer, MAX_LEN)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


***TRAINING THE SET (FITS)***

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [ ]:
from transformers import TrainerCallback

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience: int):
        self.early_stopping_patience = early_stopping_patience
        self.best_score = float('-inf')
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        eval_metrics = kwargs.get('metrics', {})
        current_score = eval_metrics.get('eval_accuracy', float('-inf'))

        if current_score > self.best_score:
            self.best_score = current_score
            self.patience_counter = 0
        else:
            self.patience_counter += 1

        if self.patience_counter >= self.early_stopping_patience:
            control.should_terminate = True

        return control

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Early stopping to avoid overfitting
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=125, training_loss=0.46063039779663084, metrics={'train_runtime': 5104.3322, 'train_samples_per_second': 0.392, 'train_steps_per_second': 0.024, 'total_flos': 263111055360000.0, 'train_loss': 0.46063039779663084, 'epoch': 5.0})

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

if 'eval_accuracy' in eval_results:
    print(f"Validation Accuracy: {eval_results['eval_accuracy']}")
else:
    print("AC not found")


Evaluation Results: {'eval_loss': 0.2704020142555237, 'eval_runtime': 74.4392, 'eval_samples_per_second': 1.343, 'eval_steps_per_second': 0.094, 'epoch': 5.0}
AC not found


**`*`FINAL RUN`*`**

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_preds = val_predictions.predictions.argmax(axis=-1)

# Compute accuracy
val_accuracy = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {val_accuracy}")

# classification report
print("\nClassification Report:")
print(classification_report(y_val, val_preds))


Validation Accuracy: 0.96

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        77
           1       0.85      1.00      0.92        23

    accuracy                           0.96       100
   macro avg       0.93      0.97      0.95       100
weighted avg       0.97      0.96      0.96       100



***APPLYING THE SAME PROCESSING TO TEST SET***

In [ ]:
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# ***`PREDICTIONS ON TEST SET`***

In [ ]:
test_dataset = TestDataset(test_df['Article'].tolist(), tokenizer, MAX_LEN)

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=-1)


**`*SUBMISSION FILE*`**

In [ ]:
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'Predicted Type': pred_labels
})

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
# from google.colab import files
# files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained("./model_path")
tokenizer.save_pretrained("./model_path")


('./model_path/tokenizer_config.json',
 './model_path/special_tokens_map.json',
 './model_path/vocab.txt',
 './model_path/added_tokens.json')

In [ ]:
!zip -r model_path.zip model_path/


  adding: model_path/ (stored 0%)
  adding: model_path/config.json (deflated 49%)
  adding: model_path/tokenizer_config.json (deflated 75%)
  adding: model_path/model.safetensors (deflated 7%)
  adding: model_path/special_tokens_map.json (deflated 42%)
  adding: model_path/vocab.txt (deflated 53%)


In [ ]:
from google.colab import files
files.download('model_path.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>